In [5]:
import os
os.chdir('..')
%pwd

'd:\\Public Projects\\ML - MLOps\\chilly_price_predictor'

In [6]:
import pandas as pd

data = pd.read_csv('artifacts\data_ingestion\Green Chilly Prices 21-25 with Dollar Rate.csv')
data.head()

,date,temp,precip,humidity,windspeed,cloudcover,dew,yesterday,today,year_month,dollar_rate
0,2021-01-01,25.0,6.778,83.6,17.3,76.7,21.8,395.0,325.0,2021-01,191.75
1,2021-01-04,25.0,16.828,93.3,13.7,95.0,23.8,325.0,365.0,2021-01,191.75
2,2021-01-05,25.5,5.183,89.8,10.4,83.9,23.6,365.0,330.0,2021-01,191.75
3,2021-01-06,26.0,7.048,87.5,6.5,77.9,23.6,330.0,295.0,2021-01,191.75
4,2021-01-07,26.2,30.440,86.2,10.8,78.4,23.5,295.0,365.0,2021-01,191.75


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: Path
    unzip_data_dir: Path
    all_schema: dict

In [8]:
from MLOps.constants import *
from MLOps.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(list_of_directories=[self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        data_validation_config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([data_validation_config.root_dir])
        return DataValidationConfig(
            root_dir=data_validation_config.root_dir,
            status_file=data_validation_config.STATUS_FILE,
            unzip_data_dir=data_validation_config.unzip_data_dir,
            all_schema=schema
        )

In [9]:
import os
from MLOps import logger

In [10]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = list(data.columns)
            
            schema = self.config.all_schema.keys()
            
            for cols in all_columns:
                if cols not in schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    break
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
            return validation_status
        except Exception as e:
            raise e
        

In [11]:
try:
    logger.info(">>>>>>>>> stage Data Validation Stage started <<<<<<<<")
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    status = data_validation.validate_all_columns()
    logger.info(f"Data validation status: {status}")
    logger.info(">>>>>>>>> stage Data Validation Stage completed <<<<<<<<")
except Exception as e:
    logger.exception(e)
    raise e

2025-07-06 22:50:50,572 - MLOpsLogger - INFO - >>>>>>>>> stage Data Validation Stage started <<<<<<<<
Directory already exists: artifacts
Created directory: artifacts\data_validation
2025-07-06 22:50:50,632 - MLOpsLogger - INFO - Data validation status: True
2025-07-06 22:50:50,634 - MLOpsLogger - INFO - >>>>>>>>> stage Data Validation Stage completed <<<<<<<<
